# OpenRouteService testing by Python Bindings

https://github.com/GIScience/openrouteservice-py

In [1]:
import openrouteservice as ors
import folium
import json
from pprint import pprint

povo_center = [46.0682, 11.1551]
povo_zoom = 16
povo_start = []

mezzo_center = [46.2147, 11.1065]
mezzo_zoom = 15
mezzo_start = [46.2170,11.1066]

#contains two features MultiPoint
povo_points = json.load(open('./data/povo.geojson'))['features']
mezzo_points = json.load(open('./data/mezzocorona.geojson'))['features']

center = mezzo_center
zoom = mezzo_zoom
points = mezzo_points


# OpenRouteService TSP Routing examples

This notebook is just a small reference on how to use `openrouteservice` package to request:
- [directions](https://openrouteservice.org/dev)
- [Route Optimization](https://openrouteservice.org/dev/#/api-docs/optimization/post)

Others examples: https://github.com/GIScience/openrouteservice-py/blob/master/test/test_optimization.py

In [2]:
M = folium.Map(location=center, tiles='OpenStreetMap', zoom_start=zoom)

client = ors.Client(base_url='http://ors:8080/ors', key='')

points = {
    'A': points[0]['geometry']['coordinates'],
    'B': points[1]['geometry']['coordinates']
}

routeA = client.directions(
    coordinates = points['A'],
    profile = 'driving-car',
    format = 'geojson',
    validate = False
)

routeB = client.directions(
    coordinates = points['B'],
    profile = 'driving-car',
    format = 'geojson',
    validate = False
)

polyA = [list(reversed(coord)) for coord in routeA['features'][0]['geometry']['coordinates']]
polyB = [list(reversed(coord)) for coord in routeB['features'][0]['geometry']['coordinates']]

folium.PolyLine(polyA, color='red').add_to(M)
folium.PolyLine(polyB, color='blue').add_to(M)
#pprint(points['A'])

for c in points['A']:
    m = list(reversed(c))
    folium.Marker(m).add_to(M)

for c in points['B']:
    m = list(reversed(c))
    folium.Marker(m).add_to(M)

M

## VRP Vehicle Routing Problem 

Optimize job scheduling for multiple vehicles.
The next example makes more use of the power of the ORS optimization endpoint. Two vehicles are assigned to 6 jobs, where the capacity constraints are such that each vehicle can only carry out 3 jobs on its route.

In [4]:
m = folium.Map(location=center, tiles='OpenStreetMap', zoom_start=zoom)

#punto di carico uguale al punto di scarico/parcheggio
start = mezzo_start
end = start
vehicle_locs = [start, end]

# veicoli aventi la stessa posizione di parteee
vehicles = []
for idx, cc in enumerate(vehicle_locs):
    loc = list(reversed(cc))
    vehicles.append(ors.optimization.Vehicle(
        id = idx,
        profile = 'driving-hgv',
        start = loc,
        end = loc,
        capacity = [3]  # Limit capacity so only 3 jobs can be taken by each vehicle
    ))
    
    folium.Marker(location=cc, icon=folium.Icon(icon='truck', prefix='fa')).add_to(m)


#punti di raccolta
job_locs = points['A'] + points['B']

jobs = []
for idx, cc in enumerate(job_locs):
    loc = list(reversed(cc))
    jobs.append(ors.optimization.Job(
        id = idx,
        location = cc,
        amount = [1] # capacity in vehicle
    ))
    folium.Marker(location=loc, icon=folium.Icon(icon='archive', prefix='fa', color='green')).add_to(m)
m

In [6]:
clientVrp = ors.Client(base_url='http://vroom:8081', key='')

optimized = clientVrp.optimization(
    jobs = jobs,
    vehicles = vehicles,
    geometry = True
)

#optimized['routes'][0]

folium.PolyLine(
    locations=[list(reversed(coords)) for coords in ors.convert.decode_polyline(optimized['routes'][0]['geometry'])['coordinates']],
    color='red'
).add_to(m)

folium.PolyLine(
    locations=[list(reversed(coords)) for coords in ors.convert.decode_polyline(optimized['routes'][1]['geometry'])['coordinates']],
    color='orange'
).add_to(m)

m